## Data Preparation

In [1]:
import pandas as pd
import json
from itertools import combinations as cb
from itertools import chain
from itertools import permutations as pm
from itertools import product as pro
import math
import random
from collections import defaultdict
from openpyxl import load_workbook
from IPython.display import display

## Select Today's team and winning percentage

In [122]:
dubai = [dub+59 for dub in [2,3,6,7,10,12,14,18,19,22,25,26,29]]
abudhabi = [ abu+59 for abu in [1,5,8,11,13,15,20,21,24,27]]
sharjah = [ shar+59 for shar in [4,9,16,17,23,28]]

stadium = abudhabi

team_num_1 = "DC"

team_num_2 = "RR"
stadium

[60, 64, 67, 70, 72, 74, 79, 80, 83, 86]

## create initial dataframe : match_df

#### 2019 IPL data

In [123]:
ls_matches  = []

for i in range(14047,14103) :
    if i == 14095:
        continue
    with open("data_extraction/Ipl-Match-{}-PlayerStats.json".format(i)) as f:
        ls_matches.append(json.load(f))

for i in range(14543,14547) :
    with open("data_extraction/Ipl-Match-{}-PlayerStats.json".format(i)) as f:
        ls_matches.append(json.load(f))
        
ls_match_df = []
ls_backup =[]

count = 1
for match in ls_matches:
    api_team1 = match['data']['site']['tour']['match']['squads'][0]['name']
    api_team1
    api_team2 = match['data']['site']['tour']['match']['squads'][1]['name']
    api_team2


    ls_players_data = match['data']['site']['tour']['match']['players']
    ls_name = []
    ls_type = []
    ls_played_for = []
    ls_played_against = []
    ls_player_score = []

    for player in ls_players_data:
        ls_name.append(player['name'])
        ls_type.append(player['type']['shortName'])
        ls_played_for.append(player['squad']['shortName'])
        if player['squad']['shortName']==api_team1:
            ls_played_against.append(api_team2)
        else:
            ls_played_against.append(api_team1)
        ls_player_score.append(pd.json_normalize(player['pointBreakup'][0]).score.tolist())

    ls_col_names = match['data']['site']['fantasyScoreCardHeader']
    ls_col = []
    for col in ls_col_names:
        ls_col.append(col['name'])
    ls_col

    ls_final = ls_played_for+ls_played_against+ls_name+ls_type+ls_player_score

    match_df = pd.DataFrame([ls_played_for,ls_played_against,ls_name,ls_type,ls_player_score]).T
    match_df.columns = ["played_for","played_against","player_name","category","score_list"]
    match_df[ls_col] = pd.DataFrame(match_df.score_list.tolist())
    match_df.drop('score_list',axis=1, inplace = True)
    match_df["match_no"] = count
    match_df["Total"] = match_df["Total"]-(match_df["Catch"]+match_df["Run Out/Stumping"]+match_df["Bonus"])
    match_df.drop(["Starting 11", "4's","6's","S/R","Duck","E/R","Bonus","Catch","Run Out/Stumping"], axis=1 , inplace =True)
    team1 = list(match_df.groupby("played_for").groups.keys())[0]
    match_df["team_no"] = match_df.played_for.map(lambda x : "team1" if x== team1 else "team2")
    ls_match_df.append(match_df[match_df.Total!=0.0].sort_values("Total",ascending =False).head(11).reset_index(drop=True))
    count+=1
matches_df = pd.concat(ls_match_df,axis=0)
matches_df[['Runs','30/50/100','Wkts','Maiden Over','Total']] *= 2
matches_df.tail(5)

,played_for,played_against,player_name,category,Runs,30/50/100,Wkts,Maiden Over,Total,match_no,team_no
6,MI,CSK,Quinton de Kock,WK,29.0,0.0,0.0,0.0,41.0,59,team2
7,CSK,MI,Faf du Plessis,BAT,26.0,0.0,0.0,0.0,35.0,59,team1
8,MI,CSK,Ishan Kishan,BAT,23.0,0.0,0.0,0.0,30.0,59,team2
9,MI,CSK,Rahul Chahar,BOWL,0.0,0.0,20.0,0.0,30.0,59,team2
10,MI,CSK,Krunal Pandya,ALL,7.0,0.0,20.0,0.0,25.0,59,team2


#### 2020 IPL data

In [124]:
ls_matches=[]
for i in range(22387,22415) :
    with open("data_extraction/Ipl-Match-{}-PlayerStats.json".format(i)) as f:
        ls_matches.append(json.load(f))
        
ls_match_df = []
ls_backup = []
for match in ls_matches:
    api_team1 = match['data']['site']['tour']['match']['squads'][0]['name']
    api_team1
    api_team2 = match['data']['site']['tour']['match']['squads'][1]['name']
    api_team2


    ls_players_data = match['data']['site']['tour']['match']['players']
    ls_name = []
    ls_type = []
    ls_played_for = []
    ls_played_against = []
    ls_player_score = []

    for player in ls_players_data:
        ls_name.append(player['name'])
        ls_type.append(player['type']['shortName'])
        ls_played_for.append(player['squad']['shortName'])
        if player['squad']['shortName']==api_team1:
            ls_played_against.append(api_team2)
        else:
            ls_played_against.append(api_team1)
        ls_player_score.append(pd.json_normalize(player['pointBreakup'][0]).score.tolist())

    ls_col_names = match['data']['site']['fantasyScoreCardHeader']
    ls_col = []
    for col in ls_col_names:
        ls_col.append(col['name'])
    ls_col

    ls_final = ls_played_for+ls_played_against+ls_name+ls_type+ls_player_score

    match_df = pd.DataFrame([ls_played_for,ls_played_against,ls_name,ls_type,ls_player_score]).T
    match_df.columns = ["played_for","played_against","player_name","category","score_list"]
    match_df[ls_col] = pd.DataFrame(match_df.score_list.tolist())
    match_df.drop('score_list',axis=1, inplace = True)
    match_df["match_no"] = count
    match_df["Total"] = match_df["Total"]-(match_df["Catch"]+match_df["Run Out/Stumping"]+match_df["Bonus"])
    match_df.drop(["Starting 11", "4's","6's","S/R","Duck","E/R","Bonus","Catch","Run Out/Stumping"], axis=1 , inplace =True)
    team1 = list(match_df.groupby("played_for").groups.keys())[0]
    match_df["team_no"] = match_df.played_for.map(lambda x : "team1" if x== team1 else "team2")
    ls_match_df.append(match_df[match_df.Total!=0.0].sort_values("Total",ascending =False).head(11).reset_index(drop=True))
    count+=1
new_df = pd.concat(ls_match_df,axis=0)
new_df.tail(5)

,played_for,played_against,player_name,category,Runs,30/50/100,Wkts,Maiden Over,Total,match_no,team_no
6,BLR,KOL,Devdutt Padikkal,BAT,32,0,0,0,42,87,team1
7,BLR,KOL,Virat Kohli,BAT,33,0,0,0,38,87,team1
8,BLR,KOL,Yuzvendra Chahal,BOWL,0,0,25,0,35,87,team1
9,BLR,KOL,Navdeep Saini,BOWL,0,0,25,0,31,87,team1
10,BLR,KOL,Mohammed Siraj,BOWL,0,0,25,0,29,87,team1


#### Combined Data set : 2019 + 2020

In [125]:
matches_df = pd.concat([matches_df,new_df],axis=0)
matches_df.tail(5)

,played_for,played_against,player_name,category,Runs,30/50/100,Wkts,Maiden Over,Total,match_no,team_no
6,BLR,KOL,Devdutt Padikkal,BAT,32.0,0.0,0.0,0.0,42.0,87,team1
7,BLR,KOL,Virat Kohli,BAT,33.0,0.0,0.0,0.0,38.0,87,team1
8,BLR,KOL,Yuzvendra Chahal,BOWL,0.0,0.0,25.0,0.0,35.0,87,team1
9,BLR,KOL,Navdeep Saini,BOWL,0.0,0.0,25.0,0.0,31.0,87,team1
10,BLR,KOL,Mohammed Siraj,BOWL,0.0,0.0,25.0,0.0,29.0,87,team1


## Preparing match_df : replace categories if category or team mis match

In [126]:
matches_df.iloc[(matches_df.player_name == 'Abhishek-Sharma').values, 2] = 'Abhishek Sharma'
matches_df.iloc[(matches_df.player_name == 'Rashid-Khan').values, 2] = 'Rashid Khan'
matches_df.iloc[(matches_df.player_name == 'Mandeep-Singh').values, 2] = 'Mandeep Singh'
matches_df.iloc[(matches_df.player_name == 'Harshal-Patel').values, 2] = 'Harshal Patel'
matches_df.iloc[(matches_df.player_name == "Axar-Patel").values, 2] = "Axar Patel"
matches_df.iloc[(matches_df.player_name == "Devdutt Padikal").values, 2] = "Devdutt Padikkal"

matches_df.iloc[(matches_df.player_name == 'Jos Buttler').values, 3] = "WK"
matches_df.iloc[(matches_df.player_name == 'Nicholas Pooran').values, 3] = "WK"
matches_df.iloc[(matches_df.player_name == 'Lokesh Rahul').values, 3] = "WK"
matches_df.iloc[(matches_df.player_name == 'Sanju Samson').values, 3] = "WK"
matches_df.iloc[(matches_df.player_name == 'Ishan Kishan').values, 3] = "WK"
matches_df.iloc[(matches_df.player_name == 'AB de Villiers').values, 3] = "WK"

matches_df.iloc[(matches_df.player_name == 'Nitish Rana').values, 3] = "BAT"
matches_df.iloc[(matches_df.player_name == 'Kedar Jadhav').values, 3] = "BAT"
matches_df.iloc[(matches_df.player_name == 'Sherfane Rutherford').values, 3] = "BAT"
matches_df.iloc[(matches_df.player_name == 'Riyan Parag').values, 3] = "BAT"
matches_df.iloc[(matches_df.player_name == 'Hardik Pandya').values, 3] = "BAT"

matches_df.iloc[(matches_df.player_name == 'Vijay Shankar').values, 3] = "ALL"
matches_df.iloc[(matches_df.player_name == 'Kieron Pollard').values, 3] = "ALL"
matches_df.iloc[(matches_df.player_name == 'Keemo Paul').values, 3] = "ALL"
matches_df.iloc[(matches_df.player_name == 'Abhishek Sharma').values, 3] = "ALL"
matches_df.iloc[(matches_df.player_name == 'Abdul Samad').values, 3] = "ALL"

matches_df.iloc[(matches_df.player_name == 'Jayant Yadav').values, 3] = "BOWL"
matches_df.iloc[(matches_df.player_name == 'Ravichandran Ashwin').values, 3] = "BOWL"

matches_df.iloc[(matches_df.played_for == "BLR").values, 0] = "RCB"
matches_df.iloc[(matches_df.played_for == "KOL").values, 0] = "KKR"
matches_df.iloc[(matches_df.played_against == "BLR").values, 1] = "RCB"
matches_df.iloc[(matches_df.played_against == "KOL").values, 1] = "KKR"

matches_df["team_cat"] = matches_df.apply(lambda x : x.team_no+"_"+x.category,axis=1)

In [127]:
wicket = matches_df[matches_df.category=="WK"].player_name
set_wicket = set(wicket.tolist())
set_wicket

{'AB de Villiers',
 'Dinesh Karthik',
 'Ishan Kishan',
 'Jonny Bairstow',
 'Jos Buttler',
 'Lokesh Rahul',
 'MS Dhoni',
 'Narayan Jagadeesan',
 'Nicholas Pooran',
 'Parthiv-Patel',
 'Quinton de Kock',
 'Rishabh Pant',
 'Sanju Samson',
 'Wriddhiman Saha'}

In [128]:
batsmen = matches_df[matches_df.category=="BAT"].player_name
set_batsmen = set(batsmen.tolist())
set_batsmen

{'Aaron Finch',
 'Ajinkya Rahane',
 'Akshdeep Nath',
 'Ambati Rayudu',
 'Chris Gayle',
 'Chris Lynn',
 'Colin Ingram',
 'Colin Munro',
 'David Miller',
 'David Warner',
 'Deepak Hooda',
 'Devdutt Padikkal',
 'Eoin Morgan',
 'Evin Lewis',
 'Faf du Plessis',
 'Gurkeerat Singh Mann',
 'Hardik Pandya',
 'Kane Williamson',
 'Kedar Jadhav',
 'Liam Livingstone',
 'Mandeep Singh',
 'Manish Pandey',
 'Martin Guptill',
 'Mayank Agarwal',
 'Murali Vijay',
 'Nitish Rana',
 'Prithvi Shaw',
 'Priyam Garg',
 'Rahul Tripathi',
 'Rinku Singh',
 'Riyan Parag',
 'Robin Uthappa',
 'Rohit Sharma',
 'Sarfaraz Khan',
 'Saurabh Tiwary',
 'Shane Watson',
 'Sherfane Rutherford',
 'Shikhar Dhawan',
 'Shimron Hetmyer',
 'Shreyas Iyer',
 'Shubman Gill',
 'Siddhesh Lad',
 'Steven Smith',
 'Suresh Raina',
 'Suryakumar Yadav',
 'Virat Kohli',
 'Yashasvi Jaiswal',
 'Yusuf Pathan',
 'Yuvraj Singh'}

In [129]:
all_rounders = matches_df[matches_df.category=="ALL"].player_name
set_all_rounders = set(all_rounders.tolist())
set_all_rounders

{'Abdul Samad',
 'Abhishek Sharma',
 'Andre Russell',
 'Anukul Sudhakar Roy',
 'Axar Patel',
 'Ben Stokes',
 'Chris Morris',
 'Colin de Grandhomme',
 'Dwayne-Bravo',
 'Glenn Maxwell',
 'Harpreet Brar',
 'Jimmy Neesham',
 'Keemo Paul',
 'Kieron Pollard',
 'Krishnappa Gowtham',
 'Krunal Pandya',
 'Mahipal Lomror',
 'Marcus Stoinis',
 'Mitchell Santner',
 'Moeen Ali',
 'Mohammad Nabi',
 'Rahul Tewatia',
 'Ravindra Jadeja',
 'Sam Curran',
 'Shakib-Al-Hasan',
 'Shivam Dube',
 'Stuart Binny',
 'Sunil Narine',
 'Vijay Shankar',
 'Washington Sundar'}

In [130]:
bowlers = matches_df[matches_df.category=="BOWL"].player_name
set_bowlers = set(bowlers.tolist())
set_bowlers

{'Alzarri Joseph',
 'Amit Mishra',
 'Andrew Tye',
 'Ankit Rajpoot',
 'Anrich Nortje',
 'Arshdeep Singh',
 'Bhuvneshwar Kumar',
 'Dale Steyn',
 'Deepak Chahar',
 'Dhawal Kulkarni',
 'Harbhajan Singh',
 'Hardus Viljoen',
 'Harry Gurney',
 'Harshal Patel',
 'Imran Tahir',
 'Ish Sodhi',
 'Ishant Sharma',
 'Isuru Udana',
 'Jagadeesha Suchith',
 'James Pattinson',
 'Jason Behrendorff',
 'Jasprit Bumrah',
 'Jayant Yadav',
 'Jaydev Unadkat',
 'Jofra Archer',
 'Kagiso Rabada',
 'Kamlesh Nagarkoti',
 'Karn Sharma',
 'Kartik Tyagi',
 'Khaleel Ahmed',
 'Kuldeep Yadav',
 'Lasith Malinga',
 'Lungi Ngidi',
 'Mitchell McClenaghan',
 'Mohammed Shami',
 'Mohammed Siraj',
 'Mujeeb-ur-Rahman',
 'Murugan Ashwin',
 'Navdeep Saini',
 'Oshane Thomas',
 'Pat Cummins',
 'Pawan Negi',
 'Piyush Chawla',
 'Prasidh Krishna',
 'Prayas Ray Barman',
 'Rahul Chahar',
 'Rashid Khan',
 'Ravi Bishnoi',
 'Ravichandran Ashwin',
 'Sandeep Lamichhane',
 'Sandeep Sharma',
 'Sandeep Warrier',
 'Scott Kuggeleijn',
 'Shardul Thak

In [131]:
set_wk_matches_df = set(matches_df[matches_df.category=="WK"].player_name.tolist())
wk = pd.read_excel("credits.xlsx", sheet_name="wk")
set_wk = set(wk.players.tolist())
print("wk : " , set_wk_matches_df.difference(set_wk))

set_bat_matches_df = set(matches_df[matches_df.category=="BAT"].player_name.tolist())
bat = pd.read_excel("credits.xlsx", sheet_name="bat")
set_bat = set(bat.players.tolist())
print("bat : " , set_bat_matches_df.difference(set_bat))

set_allr_matches_df = set(matches_df[matches_df.category=="ALL"].player_name.tolist())
allr = pd.read_excel("credits.xlsx", sheet_name="allr")
set_allr = set(allr.players.tolist())
print("allr : " , set_allr_matches_df.difference(set_allr))

set_bowl_matches_df = set(matches_df[matches_df.category=="BOWL"].player_name.tolist())
bowl = pd.read_excel("credits.xlsx", sheet_name="bowl")
set_bowl = set(bowl.players.tolist())
print("bowl : " , set_bowl_matches_df.difference(set_bowl))

wk :  set()
bat :  set()
allr :  set()
bowl :  set()


In [132]:
players_credits = pd.concat([wk,bat,allr,bowl],axis=0).reset_index(drop=True)
players_credits.head(5)

,players,credits,category
0,Dinesh Karthik,8.5,wk
1,Ishan Kishan,8.5,wk
2,Jonny Bairstow,9.5,wk
3,Jos Buttler,10.0,wk
4,Lokesh Rahul,11.0,wk


In [133]:
matches_df["sub_category"] = matches_df.apply(lambda x : players_credits[players_credits.players==x[2]].category.tolist()[0],axis=1)
matches_df["credits"] = matches_df.apply(lambda x : players_credits[players_credits.players==x[2]].credits.tolist()[0],axis=1)

## Create Data Frame for captaincy

In [134]:
total_match = matches_df.match_no.tolist()[-1]
overall_cap_df = pd.DataFrame()
count = 1
for match_no in range(1,total_match+1):
    match_df = matches_df[matches_df.match_no==match_no].copy()
    cap = pd.DataFrame()
    if match_df.iloc[0].team_no==match_df.iloc[1].team_no:
        ls_c_vc = "team_"+match_df.head(2).category
    else:
        ls_c_vc = match_df.head(2).team_no+"_"+match_df.head(2).category
    #ls_c_vc = match_df.head(2).team_no+"_"+match_df.head(2).category
    team1 = list(match_df.groupby("played_for").groups.keys())[0]
    team2 = list(match_df.groupby("played_for").groups.keys())[1]
    cap["team1"] = [team1]
    cap["team2"] = [team2]
    cap[["WK","BAT","ALL","BOWL"]] = [0,0,0,0]
    cap[['team1_WK','team1_BAT','team1_ALL', 'team1_BOWL', 'team2_WK', 'team2_BAT','team2_ALL', 'team2_BOWL']] = [0,0,0,0,0,0,0,0]
    cap[match_df.groupby("category").count()["player_name"].index.tolist()] = match_df.groupby("category").count()["player_name"].values.tolist()
    cap[match_df.groupby("team_cat").count()["player_name"].index.tolist()] = match_df.groupby("team_cat").count()["player_name"].values.tolist()
    cap.ALL = cap.ALL.astype(int)
    cap.BAT = cap.BAT.astype(int)
    cap.BOWL = cap.BOWL.astype(int)
    cap.WK = cap.WK.astype(int)
    cap.team1_ALL = cap.team1_ALL.astype(int)
    cap.team1_BAT = cap.team1_BAT.astype(int)
    cap.team1_BOWL = cap.team1_BOWL.astype(int)
    cap.team1_WK = cap.team1_WK.astype(int)
    cap.team2_ALL = cap.team2_ALL.astype(int)
    cap.team2_BAT = cap.team2_BAT.astype(int)
    cap.team2_BOWL = cap.team2_BOWL.astype(int)
    cap.team2_WK = cap.team2_WK.astype(int)
    cap["cap"] = [ls_c_vc[0]]
    cap["vice_cap"] = [ls_c_vc[1]]
    #cap = cap[["team1","team2","cap","vice_cap","WK","BAT","ALL","BOWL",'team1_WK','team1_BAT','team1_ALL', 'team1_BOWL', 'team2_WK', 'team2_BAT','team2_ALL', 'team2_BOWL']]
    cap["to_grp"] = [cap.WK.astype(str)+cap.BAT.astype(str)+cap.ALL.astype(str)+cap.BOWL.astype(str)][0]
    cap["team1_grp"] = [cap.team1_WK.astype(str)+cap.team1_BAT.astype(str)+cap.team1_ALL.astype(str)+cap.team1_BOWL.astype(str)][0]
    cap["team2_grp"] = [cap.team2_WK.astype(str)+cap.team2_BAT.astype(str)+cap.team2_ALL.astype(str)+cap.team2_BOWL.astype(str)][0]
    cap["match_no"] = count
    overall_cap_df = pd.concat([overall_cap_df,cap], axis=0)
    count+=1

overall_cap_df["cap_comb"] = overall_cap_df["cap"]+"_"+overall_cap_df["vice_cap"]
overall_cap_df["cap_comb_sort"] = overall_cap_df.apply(lambda x : ''.join(sorted(x.cap_comb)),axis=1)
backup_df = overall_cap_df.copy()
overall_cap_df = overall_cap_df[["team1","team2","cap","vice_cap","WK", 'BAT', 'ALL', 'BOWL','to_grp', 'team1_grp','team2_grp', 'match_no',"cap_comb","cap_comb_sort"]]
overall_cap_df.tail(5)

,team1,team2,cap,vice_cap,WK,BAT,ALL,BOWL,to_grp,team1_grp,team2_grp,match_no,cap_comb,cap_comb_sort
0,KKR,KXIP,team2_WK,team1_WK,3,3,1,4,3314,1211,2103,83,team2_WK_team1_WK,12KKWW___aaeemmtt
0,CSK,RCB,team_BAT,team_ALL,1,3,3,4,1334,0232,1102,84,team_BAT_team_ALL,AABLLT___aaeemmtt
0,RR,SRH,team_BAT,team_BAT,1,4,1,5,1415,1113,0302,85,team_BAT_team_BAT,AABBTT___aaeemmtt
0,DC,MI,team1_BAT,team2_WK,2,3,3,3,2333,0222,2111,86,team1_BAT_team2_WK,12ABKTW___aaeemmtt
0,KKR,RCB,team_WK,team_ALL,1,4,3,3,1433,1323,0110,87,team_WK_team_ALL,AKLLW___aaeemmtt


### Stadium DataFrame for Captaincy

In [135]:
overall_stad_cap_df = pd.DataFrame()
count = 1
for match_no in stadium:
    match_df = matches_df[matches_df.match_no==match_no].copy()
    cap = pd.DataFrame()
    if match_df.iloc[0].team_no==match_df.iloc[1].team_no:
        ls_c_vc = "team_"+match_df.head(2).category
    else:
        ls_c_vc = match_df.head(2).team_no+"_"+match_df.head(2).category
    #ls_c_vc = match_df.head(2).team_no+"_"+match_df.head(2).category
    team1 = list(match_df.groupby("played_for").groups.keys())[0]
    team2 = list(match_df.groupby("played_for").groups.keys())[1]
    cap["team1"] = [team1]
    cap["team2"] = [team2]
    cap[["WK","BAT","ALL","BOWL"]] = [0,0,0,0]
    cap[['team1_WK','team1_BAT','team1_ALL', 'team1_BOWL', 'team2_WK', 'team2_BAT','team2_ALL', 'team2_BOWL']] = [0,0,0,0,0,0,0,0]
    cap[match_df.groupby("category").count()["player_name"].index.tolist()] = match_df.groupby("category").count()["player_name"].values.tolist()
    cap[match_df.groupby("team_cat").count()["player_name"].index.tolist()] = match_df.groupby("team_cat").count()["player_name"].values.tolist()
    cap.ALL = cap.ALL.astype(int)
    cap.BAT = cap.BAT.astype(int)
    cap.BOWL = cap.BOWL.astype(int)
    cap.WK = cap.WK.astype(int)
    cap.team1_ALL = cap.team1_ALL.astype(int)
    cap.team1_BAT = cap.team1_BAT.astype(int)
    cap.team1_BOWL = cap.team1_BOWL.astype(int)
    cap.team1_WK = cap.team1_WK.astype(int)
    cap.team2_ALL = cap.team2_ALL.astype(int)
    cap.team2_BAT = cap.team2_BAT.astype(int)
    cap.team2_BOWL = cap.team2_BOWL.astype(int)
    cap.team2_WK = cap.team2_WK.astype(int)
    cap["cap"] = [ls_c_vc[0]]
    cap["vice_cap"] = [ls_c_vc[1]]
    #cap = cap[["team1","team2","cap","vice_cap","WK","BAT","ALL","BOWL",'team1_WK','team1_BAT','team1_ALL', 'team1_BOWL', 'team2_WK', 'team2_BAT','team2_ALL', 'team2_BOWL']]
    cap["to_grp"] = [cap.WK.astype(str)+cap.BAT.astype(str)+cap.ALL.astype(str)+cap.BOWL.astype(str)][0]
    cap["team1_grp"] = [cap.team1_WK.astype(str)+cap.team1_BAT.astype(str)+cap.team1_ALL.astype(str)+cap.team1_BOWL.astype(str)][0]
    cap["team2_grp"] = [cap.team2_WK.astype(str)+cap.team2_BAT.astype(str)+cap.team2_ALL.astype(str)+cap.team2_BOWL.astype(str)][0]
    cap["match_no"] = count
    overall_stad_cap_df = pd.concat([overall_stad_cap_df,cap], axis=0)
    count+=1

overall_stad_cap_df["cap_comb"] = overall_stad_cap_df["cap"]+"_"+overall_stad_cap_df["vice_cap"]
overall_stad_cap_df["cap_comb_sort"] = overall_stad_cap_df.apply(lambda x : ''.join(sorted(x.cap_comb)),axis=1)
backup__stad_df = overall_stad_cap_df.copy()
overall_stad_cap_df = overall_stad_cap_df[["team1","team2","cap","vice_cap","WK", 'BAT', 'ALL', 'BOWL','to_grp', 'team1_grp','team2_grp', 'match_no',"cap_comb","cap_comb_sort"]]
overall_stad_cap_df.tail(5)

,team1,team2,cap,vice_cap,WK,BAT,ALL,BOWL,to_grp,team1_grp,team2_grp,match_no,cap_comb,cap_comb_sort
0,RCB,RR,team_BAT,team_BAT,1,3,2,5,1325,0203,1122,6,team_BAT_team_BAT,AABBTT___aaeemmtt
0,MI,RR,team_BOWL,team_BAT,2,3,0,6,2306,1304,1002,7,team_BOWL_team_BAT,ABBLOTW___aaeemmtt
0,CSK,KKR,team2_BAT,team1_ALL,0,3,5,3,0353,0232,0121,8,team2_BAT_team1_ALL,12AABLLT___aaeemmtt
0,KKR,KXIP,team2_WK,team1_WK,3,3,1,4,3314,1211,2103,9,team2_WK_team1_WK,12KKWW___aaeemmtt
0,DC,MI,team1_BAT,team2_WK,2,3,3,3,2333,0222,2111,10,team1_BAT_team2_WK,12ABKTW___aaeemmtt


## Team1

In [136]:
team_num_1

'DC'

In [137]:
ls_players_team1 = pd.read_excel("player_selection.xlsx",sheet_name=team_num_1, usecols = ["player_name"]).dropna().player_name.tolist()

ls_players_team1

['Rishabh Pant',
 'Shikhar Dhawan',
 'Shreyas Iyer',
 'Prithvi Shaw',
 'Shimron Hetmyer',
 'Marcus Stoinis',
 'Kagiso Rabada',
 'Ravichandran Ashwin',
 'Axar Patel',
 'Anrich Nortje',
 'Harshal Patel']

In [138]:
team1_top_players_df = matches_df[matches_df.player_name.isin(ls_players_team1)].groupby('player_name').count().sort_values("played_for",ascending=False)
team1_player_quantile = team1_top_players_df.played_for.quantile(0.5)
print(team1_top_players_df[team1_top_players_df.played_for>=team1_player_quantile].index.tolist())

['Kagiso Rabada', 'Shikhar Dhawan', 'Rishabh Pant', 'Axar Patel', 'Shreyas Iyer', 'Ravichandran Ashwin']


In [139]:
team1_top_players_df = matches_df[(matches_df.player_name.isin(ls_players_team1)) & matches_df.match_no.isin(stadium)].groupby('player_name').count().sort_values("played_for",ascending=False)
team1_player_quantile = team1_top_players_df.played_for.quantile(0.6)
print(team1_top_players_df[team1_top_players_df.played_for>=team1_player_quantile].index.tolist())

['Kagiso Rabada', 'Shikhar Dhawan', 'Axar Patel', 'Marcus Stoinis', 'Ravichandran Ashwin', 'Rishabh Pant', 'Shimron Hetmyer', 'Shreyas Iyer']


## Team2

In [140]:
team_num_2

'RR'

In [141]:
ls_players_team2 = pd.read_excel("player_selection.xlsx",sheet_name=team_num_2, usecols = ["player_name"]).dropna().player_name.tolist()

ls_players_team2

['Yashasvi Jaiswal',
 'Ankit Rajpoot',
 'Sanju Samson',
 'Jos Buttler',
 'Steven Smith',
 'Rahul Tewatia',
 'Mahipal Lomror',
 'Jofra Archer',
 'Shreyas Gopal',
 'Tom Curran',
 'Kartik Tyagi',
 'Jaydev Unadkat']

In [142]:
team2_top_players_df = matches_df[matches_df.player_name.isin(ls_players_team2)].groupby('player_name').count().sort_values("played_for",ascending=False)
team2_player_quantile = team2_top_players_df.played_for.quantile(0.5)
print(team2_top_players_df[team2_top_players_df.played_for>=team2_player_quantile].index.tolist())

['Jofra Archer', 'Shreyas Gopal', 'Sanju Samson', 'Rahul Tewatia', 'Steven Smith', 'Jos Buttler']


In [143]:
team2_top_players_df = matches_df[(matches_df.player_name.isin(ls_players_team2)) & matches_df.match_no.isin(stadium)].groupby('player_name').count().sort_values("played_for",ascending=False)
team2_player_quantile = team2_top_players_df.played_for.quantile(0.6)
print(team2_top_players_df[team2_top_players_df.played_for>=team2_player_quantile].index.tolist())

['Jofra Archer', 'Jos Buttler', 'Shreyas Gopal']


### Average Team Combinations

In [144]:
try_df = backup_df[((backup_df.team1==team_num_1) | (backup_df.team2==team_num_1))].reset_index(drop=True)
try_df_1 = try_df.apply(lambda x: x if x.team1==team_num_1 else None, axis=1,result_type="broadcast").dropna()[["team2","team1_WK","team1_BAT","team1_ALL","team1_BOWL"]]
ls_try_df_1_index = try_df_1.index.tolist()
try_df_1.columns = ['played_against','WK','BAT','ALL','BOWL']
try_df_2 = try_df[~try_df.index.isin(ls_try_df_1_index)][["team1","team2_WK","team2_BAT","team2_ALL","team2_BOWL"]]
try_df_2.columns = ['played_against','WK','BAT','ALL','BOWL']
if len(try_df_1)==0:
    team1_against_others_df=try_df_2
elif len(try_df_2)==0:
    team1_against_others_df=try_df_1
else:
    team1_against_others_df = pd.concat([try_df_1,try_df_2],axis=0).dropna()
team1_against_others_df['WK'] = team1_against_others_df.WK.astype(int)
team1_against_others_df['BAT'] = team1_against_others_df.BAT.astype(int)
team1_against_others_df['ALL'] = team1_against_others_df.ALL.astype(int)
team1_against_others_df['BOWL'] = team1_against_others_df.BOWL.astype(int)
team1_avg_df = round(pd.DataFrame(team1_against_others_df.mean(axis=0, numeric_only=True)).T,2)



try_df = backup_df[((backup_df.team1==team_num_2) | (backup_df.team2==team_num_2))].reset_index(drop=True)
try_df_1 = try_df.apply(lambda x: x if x.team1==team_num_2 else None, axis=1,result_type="broadcast").dropna()[["team2","team1_WK","team1_BAT","team1_ALL","team1_BOWL"]]
ls_try_df_1_index = try_df_1.index.tolist()
try_df_1.columns = ['played_against','WK','BAT','ALL','BOWL']
try_df_2 = try_df[~try_df.index.isin(ls_try_df_1_index)][["team1","team2_WK","team2_BAT","team2_ALL","team2_BOWL"]]
try_df_2.columns = ['played_against','WK','BAT','ALL','BOWL']
if len(try_df_1)==0:
    team2_against_others_df=try_df_2
elif len(try_df_2)==0:
    team2_against_others_df=try_df_1
else:
    team2_against_others_df = pd.concat([try_df_1,try_df_2],axis=0).dropna()
team2_against_others_df['WK'] = team2_against_others_df.WK.astype(int)
team2_against_others_df['BAT'] = team2_against_others_df.BAT.astype(int)
team2_against_others_df['ALL'] = team2_against_others_df.ALL.astype(int)
team2_against_others_df['BOWL'] = team2_against_others_df.BOWL.astype(int)
team2_avg_df = round(pd.DataFrame(team2_against_others_df.mean(axis=0, numeric_only=True)).T,2)


In [145]:
try_df = backup_df[((backup_df.team1==team_num_2) | (backup_df.team2==team_num_2))].reset_index(drop=True)
try_df_1 = try_df.apply(lambda x: x[["team2","team1_WK","team1_BAT","team1_ALL","team1_BOWL"]] if x.team1==team_num_2 else None, axis=1).dropna()
ls_try_df_1_index = try_df_1.index.tolist()
try_df_1.columns = ['played_against','WK','BAT','ALL','BOWL']
try_df_2 = try_df[~try_df.index.isin(ls_try_df_1_index)][["team1","team2_WK","team2_BAT","team2_ALL","team2_BOWL"]]
try_df_2.columns = ['played_against','WK','BAT','ALL','BOWL']
if len(try_df_1)==0:
    team2_against_others_df=try_df_2
elif len(try_df_2)==0:
    team2_against_others_df=try_df_1

t1_against_t2 = pd.DataFrame(round(team1_against_others_df[team1_against_others_df.played_against==team_num_2].mean(axis=0,numeric_only=True),2)).T

t2_against_t1 = pd.DataFrame(round(team2_against_others_df[team2_against_others_df.played_against==team_num_1].mean(axis=0,numeric_only=True),2)).T

In [150]:
overall_avg_df = round(pd.DataFrame(overall_cap_df.mean(axis=0, numeric_only=True)).T,2).iloc[:,:-1]

overall_team1_team2_df = overall_cap_df[((overall_cap_df.team1==team_num_1) & (overall_cap_df.team2==team_num_2)) | ((overall_cap_df.team1==team_num_2) & (overall_cap_df.team2==team_num_1))].copy()
team1_team2_avg_df = round(pd.DataFrame(overall_team1_team2_df.mean(axis=0, numeric_only=True)).T,2).iloc[:,:-1]

In [151]:
overall_stad_avg_df = round(pd.DataFrame(overall_cap_df.mean(axis=0, numeric_only=True)).T,2).iloc[:,:-1]

overall_team1_team2_df = overall_cap_df[((overall_cap_df.team1==team_num_1) & (overall_cap_df.team2==team_num_2)) | ((overall_cap_df.team1==team_num_2) & (overall_cap_df.team2==team_num_1))].copy()
team1_team2_stad_avg_df = round(pd.DataFrame(overall_team1_team2_df.mean(axis=0, numeric_only=True)).T,2).iloc[:,:-1]

### Refer Below to choose team

In [152]:
overall_team1_team2_df

,team1,team2,cap,vice_cap,WK,BAT,ALL,BOWL,to_grp,team1_grp,team2_grp,match_no,cap_comb,cap_comb_sort
0,DC,RR,team2_BAT,team1_WK,1,5,2,3,1523,1212,0311,40,team2_BAT_team1_WK,12ABKTW___aaeemmtt
0,DC,RR,team2_BOWL,team1_WK,1,5,0,5,1505,1303,0202,52,team2_BOWL_team1_WK,12BKLOWW___aaeemmtt
0,DC,RR,team1_ALL,team2_BOWL,0,3,3,5,0335,0123,0212,82,team1_ALL_team2_BOWL,12ABLLLOW___aaeemmtt


In [153]:
team_combinations = pd.concat([overall_avg_df,team1_avg_df,team2_avg_df,team1_team2_avg_df,t1_against_t2,t2_against_t1],axis=0)
team_combinations.index = ["overall",team_num_1+" vs Others",team_num_2+" v s Others",team_num_1+" vs "+team_num_2,team_num_1+" agnst "+team_num_2 ,team_num_2+" agnst "+team_num_1]
team_combinations

,WK,BAT,ALL,BOWL
overall,1.47,3.64,2.03,3.85
DC vs Others,0.57,1.96,1.09,1.87
RR vs Others,0.85,1.45,0.70,2.00
DC vs RR,0.67,4.33,1.67,4.33
DC agnst RR,0.67,2.00,1.00,2.67
RR agnst DC,0.00,2.33,0.67,1.67


In [120]:
team_combinations = pd.concat([overall_avg_df,team1_avg_df,team2_avg_df,team1_team2_avg_df,t1_against_t2,t2_against_t1],axis=0)
team_combinations.index = ["overall",team_num_1+" vs Others",team_num_2+" vs Others",team_num_1+" vs "+team_num_2,team_num_1+" agnst "+team_num_2 ,team_num_2+" agnst "+team_num_1]
team_combinations

,WK,BAT,ALL,BOWL
overall,1.47,3.64,2.03,3.85
SRH vs Others,0.64,1.73,0.86,2.32
CSK vs Others,0.42,2.17,1.33,2.00
SRH vs CSK,0.67,4.67,1.67,4.00
SRH agnst CSK,0.33,2.00,1.33,2.00
CSK agnst SRH,0.33,2.67,0.33,2.00


In [27]:
match_1 = matches_df[matches_df.player_name.isin(ls_players_team1)].groupby(["player_name",'category']).sum(numeric_only=True).sort_values("Total", ascending = False).head(18).reset_index().iloc[:,:-2]
match_1["team"] = team_num_1
match_2 = matches_df[matches_df.player_name.isin(ls_players_team2)].groupby(["player_name",'category']).sum(numeric_only=True).sort_values("Total", ascending = False).head(18).reset_index().iloc[:,:-2]
match_2["team"] = team_num_2
pd.concat([match_1,match_2],axis=1)

,player_name,category,Runs,30/50/100,Wkts,Maiden Over,Total,team,player_name,category,Runs,30/50/100,Wkts,Maiden Over,Total,team
0,David Warner,BAT,888.0,96.0,0.0,0.0,1171.0,SRH,Faf du Plessis,BAT,659.0,48.0,0.0,0.0,866.0,CSK
1,Jonny Bairstow,WK,639.0,56.0,0.0,0.0,849.0,SRH,Shane Watson,BAT,510.0,40.0,0.0,0.0,695.0,CSK
2,Rashid Khan,BOWL,33.0,0.0,475.0,16.0,616.0,SRH,Deepak Chahar,BOWL,12.0,0.0,570.0,16.0,671.0,CSK
3,Manish Pandey,BAT,444.0,32.0,0.0,0.0,583.0,SRH,Ravindra Jadeja,ALL,164.0,8.0,375.0,8.0,646.0,CSK
4,Sandeep Sharma,BOWL,5.0,0.0,290.0,0.0,322.0,SRH,MS Dhoni,WK,453.0,24.0,0.0,0.0,594.0,CSK
5,Kane Williamson,BAT,173.0,8.0,0.0,0.0,226.0,SRH,Sam Curran,ALL,142.0,8.0,375.0,0.0,581.0,CSK
6,T Natarajan,BOWL,3.0,0.0,175.0,0.0,192.0,SRH,Ambati Rayudu,BAT,387.0,16.0,0.0,0.0,491.0,CSK
7,Siddarth Kaul,BOWL,0.0,0.0,150.0,0.0,160.0,SRH,Dwayne-Bravo,ALL,64.0,0.0,295.0,0.0,397.0,CSK
8,Abhishek Sharma,ALL,52.0,0.0,70.0,0.0,148.0,SRH,Shardul Thakur,BOWL,3.0,0.0,310.0,0.0,331.0,CSK
9,Priyam Garg,BAT,51.0,8.0,0.0,0.0,71.0,SRH,Kedar Jadhav,BAT,166.0,8.0,0.0,0.0,220.0,CSK


In [28]:
google_sheet = pd.concat([match_1,match_2],axis=0)
google_sheet[(google_sheet.category=="WK") & (google_sheet.team==team_num_1)].player_name.tolist()

['Jonny Bairstow']

In [29]:
google_sheet[(google_sheet.category=="WK") & (google_sheet.team==team_num_2)].player_name.tolist()

['MS Dhoni']

In [30]:
google_sheet[(google_sheet.category=="BAT") & (google_sheet.team==team_num_1)].player_name.tolist()

['David Warner', 'Manish Pandey', 'Kane Williamson', 'Priyam Garg']

In [31]:
google_sheet[(google_sheet.category=="BAT") & (google_sheet.team==team_num_2)].player_name.tolist()

['Faf du Plessis', 'Shane Watson', 'Ambati Rayudu', 'Kedar Jadhav']

In [32]:
google_sheet[(google_sheet.category=="ALL") & (google_sheet.team==team_num_1)].player_name.tolist()

['Abhishek Sharma', 'Abdul Samad']

In [33]:
google_sheet[(google_sheet.category=="ALL") & (google_sheet.team==team_num_2)].player_name.tolist()

['Ravindra Jadeja', 'Sam Curran', 'Dwayne-Bravo']

In [34]:
google_sheet[(google_sheet.category=="BOWL") & (google_sheet.team==team_num_1)].player_name.tolist()

['Rashid Khan', 'Sandeep Sharma', 'T Natarajan', 'Siddarth Kaul']

In [35]:
google_sheet[(google_sheet.category=="BOWL") & (google_sheet.team==team_num_2)].player_name.tolist()

['Deepak Chahar', 'Shardul Thakur']

In [36]:
print(team1_top_players_df[team1_top_players_df.played_for>=team1_player_quantile].index.tolist())
print(team2_top_players_df[team2_top_players_df.played_for>=team2_player_quantile].index.tolist())

['David Warner', 'Rashid Khan', 'Jonny Bairstow', 'Manish Pandey', 'Sandeep Sharma', 'Kane Williamson', 'T Natarajan']
['Deepak Chahar', 'Ravindra Jadeja', 'Faf du Plessis', 'Ambati Rayudu', 'MS Dhoni', 'Shane Watson']


In [100]:
def update_excel(df,team_for,scenario_num, row, col):
    writer = pd.ExcelWriter("send/{}.xlsx".format(team_for), engine='openpyxl')
    writer.book = load_workbook("send/{}.xlsx".format(team_for))
    writer.sheets = dict((ws.title, ws) for ws in writer.book.worksheets)
    df.to_excel(writer, sheet_name = "scenario{}".format(scenario_num), index = False,startrow=row, startcol=col)
    writer.close()
    
def selected_func(team1_priority1,team2_priority1,matches_df):
    scene = set(team1_priority1+team2_priority1)  
    selected = matches_df[matches_df.player_name.isin(scene)].drop_duplicates(subset=['player_name'])[['played_for','player_name','category','credits']].reset_index(drop=True)
    selected.columns = ['team','player','category','credits']
    selected.sort_values(["team","credits"], inplace=True)
    selected.reset_index(drop=True,inplace=True)
    set_diff = set(scene).difference(set(selected.player.tolist()))
    
    print("Players not present in selected dataframe : ",set_diff)
    
    wrong_team = selected[(selected.team!=team_num_1) & (selected.team!=team_num_2)][["team","player"]].set_index("player").to_dict()["team"]
    if len(wrong_team.values())>0:
        for player,team in wrong_team.items():
            if player in team1_priority1:
                selected.iloc[(selected.team==team).values,0] = team_num_1
            else:
                selected.iloc[(selected.team==team).values,0] = team_num_2
                
    return selected, list(set_diff)

def tot_player(x):
    return set(chain(*x.unique()))

def player_count(selected,x,flat_team1_mem,ls_team1,ls_team2,wk,bat,allr,bowl,ls_wk,ls_bat,ls_allr,ls_bowl,ls_credits):
    count_t1 = 0
    count_t2 = 0

    wk_count = 0
    bat_count = 0
    allr_count = 0
    bowl_count = 0
    
    sort_wk = []
    sort_bat = []
    sort_allr = []
    sort_bowl = []
    
    
    
    for player in x:
        if player in flat_team1_mem:
            count_t1+=1
        else:
            count_t2+=1
        
        if player in wk:
            sort_wk.append(player)
            wk_count+=1
        elif player in bat:
            sort_bat.append(player)
            bat_count+=1
        elif player in allr:
            sort_allr.append(player)
            allr_count+=1
        else:
            sort_bowl.append(player)
            bowl_count+=1
            
            
    ls_team1.append(count_t1)
    ls_team2.append(count_t2)
    
    ls_wk.append(wk_count)
    ls_bat.append(bat_count)
    ls_allr.append(allr_count)
    ls_bowl.append(bowl_count)
    
    ls_credits.append(selected[selected.player.isin(x)].credits.sum())   
        
    
    return sort_wk+sort_bat+sort_allr+sort_bowl
    
def wk_bat_all_bowl_comb(team_num_1, selected, min_wk, max_wk, min_bat, max_bat, min_allr, max_allr, min_bowl, max_bowl):
    wickt_keepers =  selected[selected.category=="WK"]["player"].tolist()
    batsmen = selected[selected.category=="BAT"]["player"].tolist()
    bowler = selected[selected.category=="BOWL"]["player"].tolist()
    all_rounder = selected[selected.category=="ALL"]["player"].tolist()

    ls = []
    for i in range(min_wk,max_wk+1):
        ls+=list(cb(wickt_keepers,i))
    wk_comb = ls
    
    ls = []
    for i in range(min_bat,max_bat+1):
        ls+=list(cb(batsmen,i))
    bat_comb = ls

    ls = []
    for i in range(min_bowl,max_bowl+1):
        ls+=list(cb(bowler,i))
    bowl_comb = ls

    ls = []
    for i in range(min_allr,max_allr+1):
        ls+=list(cb(all_rounder,i))
    all_comb = ls

    tot = wk_comb+bat_comb+bowl_comb+all_comb
    ls_tot_comb = list(cb(tot,4))
    print("Total Random Teams Created : ",len(ls_tot_comb))
    
    tot_df = pd.DataFrame(ls_tot_comb)
    player_df = pd.DataFrame(tot_df.apply(lambda x : tot_player(x), axis=1))
    player_df.columns = ["players"]

    player_df["len"] = player_df.players.map(lambda x : len(x))
    player_df.drop(player_df[player_df.len!=11].index,axis=0, inplace= True)
    player_df.reset_index(inplace=True, drop=True)

    flat_team1_mem = set(selected[selected.team==team_num_1].player.tolist())

    ls_team1 = []
    ls_team2 = []
    df = player_df.copy()
    
    ls_wk = []
    ls_bat = []
    ls_bowl = []
    ls_allr = []
    ls_credits = []
    
    wk = set(selected[selected.category=="WK"].player.tolist())
    bat = set(selected[selected.category=="BAT"].player.tolist())
    allr = set(selected[selected.category=="ALL"].player.tolist())
    bowl = set(selected[selected.category=="BOWL"].player.tolist())
    
    
    df['players'] = df.players.apply(lambda x : player_count(selected,x,flat_team1_mem,ls_team1,ls_team2,wk,bat,allr,bowl,ls_wk,ls_bat,ls_allr,ls_bowl,ls_credits))

    df["team1"] = ls_team1
    df["team2"] = ls_team2
    df['total'] = df['team1']+df['team2']
    df["wk"] = ls_wk
    df["bat"] = ls_bat
    df["allr"] = ls_allr
    df["bowl"] = ls_bowl

    df["credits"] = ls_credits
    df.drop(['len'],axis=1,inplace=True)
    df = df[((df.wk>=min_wk) & (df.wk<=max_wk)) & ((df.bat>=min_bat) & (df.bat<=max_bat)) & ((df.bowl>=min_bowl) & (df.bowl<=max_bowl)) & ((df.allr>=min_allr) & (df.allr<=max_allr)) & (df.team1<8) & (df.team2<8) & (df.credits<=100)].sort_values("credits",ascending = False).reset_index(drop=True)
    
    display(df)
    

      
    try:
        send_excel_df = pd.DataFrame()
        send_excel_df['players'] = df["players"]    
        send_excel_df[["player1","player2","player3","player4","player5","player6","player7","player8","player9","player10","player11"]] = pd.DataFrame(send_excel_df.players.tolist())
        send_excel_df.drop(['players'],inplace=True, axis=1)

        send_excel_df['team1'] = df["team1"]
        send_excel_df['team2'] = df["team2"]
        send_excel_df['wk'] = df["wk"]
        send_excel_df['bat'] = df["bat"]
        send_excel_df['allr'] = df["allr"]
        send_excel_df['bowl'] = df["bowl"]
        send_excel_df['credits'] = df["credits"]

        return send_excel_df, df

    except:
        print("exception created.  Total no. of teams created : ",len(send_excel_df)) 
        return pd.DataFrame(), pd.DataFrame()
    
def create_send_excel_df(ls_captaincy,team1_priority1,team2_priority1,matches_df,team_for,scenario_num):
    min_wk = 1
    max_wk = 4

    min_bat = 3
    max_bat = 5

    min_allr = 1
    max_allr = 3

    min_bowl = 3
    max_bowl = 5
    
    if (scenario_num==1) | (scenario_num==4) | (scenario_num==5):
        max_bowl=4
        min_bat=4
    ## Create Selected DataFrame
    selected, player_diff = selected_func(team1_priority1,team2_priority1,matches_df)
    if len(player_diff)>0:
        ls_team_name = []
        ls_player_name = []
        ls_category = []
        ls_credits = []
        for i in range(len(player_diff)):
            ls_team_name.append(str(input("Enter team_name for {}: ".format(player_diff[i]))).upper())
            ls_player_name.append(player_diff[i])
            ls_category.append(str(input("Enter category for {}: ".format(player_diff[i]))).upper())
            ls_credits.append(float(input("Enter credits for {}: ".format(player_diff[i]))))

        exclude = pd.DataFrame([ls_team_name,ls_player_name,ls_category,ls_credits]).T
        exclude.columns = selected.columns
        selected = pd.concat([selected,exclude],axis=0)
        selected.sort_values("player",inplace=True)
        selected.reset_index(drop=True,inplace=True)    
    print("Selected Players DataFrame for current scenario : ")
    display(selected)
    
    selected.sort_values(["team","category"]).to_excel("send/{}.xlsx".format(team_for),index= False, sheet_name="scenario{}".format(scenario_num))
    
    captaincy_df = pd.DataFrame(ls_captaincy,columns = ["captains"])
    update_excel(captaincy_df,team_for,scenario_num,0,len(selected.columns)+2)
    
    # Check Total players based on category
    print("min_wk: {}, max_wk: {}".format(min_wk,max_wk))
    print("min_bat: {}, max_bat: {}".format(min_bat,max_bat))
    print("min_allr: {}, max_allr: {}".format(min_allr,max_allr))
    print("min_bowl: {}, max_bowl: {}".format(min_bowl,max_bowl))
    display(selected.groupby("category").count()[["player"]].T[["WK","BAT","ALL","BOWL"]])

    # Create df to be sent
    send_excel_df,df = wk_bat_all_bowl_comb(team_num_1, selected, min_wk, max_wk, min_bat, max_bat, min_allr, max_allr, min_bowl, max_bowl)
        
    #if scenario_num==3:
        #send_excel_df =  send_excel_df[(send_excel_df.team1<7)&(send_excel_df.team2<6)].reset_index(drop=True)
        #df = df[(df.team1<7)&(df.team2<6)]
    
    if len(df)==0:
        print("\n\nLength of df is 0 so re calculating...\n")
        send_excel_df,df = wk_bat_all_bowl_comb(team_num_1, selected, 1, 4, 3, 6, 1, 4, 3, 6)
    frequ = pd.DataFrame()
    frequ['players'] = df.players.tolist()
    frequ['players'] = frequ.apply(lambda x : list(x[0]),axis=1)
    ls_players = list(chain(*frequ.players.tolist()))
    
    freq_dic = defaultdict(int) 
    for i in ls_players:
        freq_dic[i]+=1
    freq_tuple = list(freq_dic.items())
    freq_df = pd.DataFrame(freq_tuple).rename(columns = {0:"player",1:"frequency"}).sort_values("frequency",ascending = False).reset_index(drop=True)
    
    print("Frequency of players in the current data frame : ")
    display(freq_df)
    
    print("Total Number of Teams send_excel_df : ",len(send_excel_df))
    display(send_excel_df)
    
    update_excel(freq_df,team_for,scenario_num,0,len(selected.columns)+2+len(captaincy_df.columns)+2)
    update_excel(send_excel_df,team_for,scenario_num,len(selected)+2,0)
    
    return send_excel_df

In [101]:
pradeep = "Pradeep openers_cap_{}".format(team_num_1)
mazeen = "Mazeen openers_failed_cap_{}".format(team_num_1)
sughosh = "Sughosh {} openers_{} middle".format(team_num_1,team_num_2)
sharath = "Sharath openers_cap_{}".format(team_num_2)
nagaraj = "Nagaraj openers_cap_{}".format(team_num_2)
nishant = "Nishant {} middle_{} openers".format(team_num_1,team_num_2)

teams_for = [pradeep,mazeen,sughosh,sharath,nagaraj,nishant]#,jeson

ls_send_excel_df = []

total_scenarios = len(teams_for)

count = 1
for i in range(total_scenarios) :  
    print("Showing details for Scenairo {}\n\n".format(count))
    
    scenario = "scenario{}".format(count)
    team1_df = pd.read_excel("player_selection.xlsx",sheet_name=team_num_1, usecols = ["player_name",scenario]).dropna()
    team1_priority1 = team1_df.player_name.tolist()
    captaincy_team1 = team1_df[team1_df[scenario]==2].player_name.tolist()

    team2_df = pd.read_excel("player_selection.xlsx",sheet_name=team_num_2, usecols = ["player_name",scenario]).dropna()
    team2_priority1 = team2_df.player_name.tolist()
    captaincy_team2 = team2_df[team2_df[scenario]==2].player_name.tolist()

    scenes = team1_priority1+team2_priority1
    captains = captaincy_team1+captaincy_team2
    
    for player in team1_priority1:
        matches_df.iloc[(matches_df.player_name == player).values, 0] = team_num_1
        
    for player in team2_priority1:
        matches_df.iloc[(matches_df.player_name == player).values, 0] = team_num_2
    
    ls_send_excel_df.append(create_send_excel_df(captains,team1_priority1,team2_priority1,matches_df,teams_for[count-1],count))
    update_excel(team_combinations.reset_index().rename(columns = {"index":"stats"}),teams_for[count-1],count,0,13)
    count+=1

Showing details for Scenairo 1


Players not present in selected dataframe :  set()
Selected Players DataFrame for current scenario : 


,team,player,category,credits
0,RR,Yashasvi Jaiswal,BAT,8.0
1,RR,Rahul Tewatia,ALL,8.5
2,RR,Shreyas Gopal,BOWL,8.5
3,RR,Jofra Archer,BOWL,9.0
4,RR,Sanju Samson,WK,9.5
5,RR,Steven Smith,BAT,9.5
6,RR,Jos Buttler,WK,10.0
7,SRH,Abhishek Sharma,ALL,8.0
8,SRH,T Natarajan,BOWL,8.5
9,SRH,Manish Pandey,BAT,9.0


min_wk: 1, max_wk: 4
min_bat: 4, max_bat: 5
min_allr: 1, max_allr: 3
min_bowl: 3, max_bowl: 4


category,WK,BAT,ALL,BOWL
player,3,5,2,4


Total Random Teams Created :  5985


...

,player1,player2,player3,player4,player5,player6,player7,player8,player9,player10,player11,team1,team2,wk,bat,allr,bowl,credits
0,Jos Buttler,Jonny Bairstow,Sanju Samson,Manish Pandey,Kane Williamson,David Warner,Yashasvi Jaiswal,Abhishek Sharma,Shreyas Gopal,Jofra Archer,T Natarajan,6,5,3,4,1,3,100.0
1,Jonny Bairstow,Sanju Samson,Manish Pandey,Kane Williamson,David Warner,Yashasvi Jaiswal,Rahul Tewatia,Shreyas Gopal,Rashid Khan,Jofra Archer,T Natarajan,6,5,2,4,1,4,100.0
2,Jos Buttler,Sanju Samson,Manish Pandey,David Warner,Steven Smith,Yashasvi Jaiswal,Rahul Tewatia,Abhishek Sharma,Rashid Khan,Jofra Archer,T Natarajan,5,6,2,4,2,3,100.0
3,Jos Buttler,Sanju Samson,Manish Pandey,David Warner,Steven Smith,Yashasvi Jaiswal,Abhishek Sharma,Shreyas Gopal,Rashid Khan,Jofra Archer,T Natarajan,5,6,2,4,1,4,100.0
4,Jos Buttler,Sanju Samson,Kane Williamson,David Warner,Steven Smith,Yashasvi Jaiswal,Rahul Tewatia,Abhishek Sharma,Shreyas Gopal,Rashid Khan,T Natarajan,5,6,2,4,2,3,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,Jos Buttler,Sanju Samson,Manish Pandey,Kane Williamson,Steven Smith,Yashasvi Jaiswal,Rahul Tewatia,Abhishek Sharma,Shreyas Gopal,Jofra Archer,T Natarajan,4,7,2,4,2,3,98.0
100,Jos Buttler,Manish Pandey,Kane Williamson,Steven Smith,Yashasvi Jaiswal,Rahul Tewatia,Abhishek Sharma,Shreyas Gopal,Rashid Khan,Jofra Archer,T Natarajan,5,6,1,4,2,4,98.0
101,Jonny Bairstow,Sanju Samson,Manish Pandey,Kane Williamson,Steven Smith,Yashasvi Jaiswal,Rahul Tewatia,Abhishek Sharma,Shreyas Gopal,Jofra Archer,T Natarajan,5,6,2,4,2,3,97.5
102,Jonny Bairstow,Manish Pandey,Kane Williamson,Steven Smith,Yashasvi Jaiswal,Rahul Tewatia,Abhishek Sharma,Shreyas Gopal,Rashid Khan,Jofra Archer,T Natarajan,6,5,1,4,2,4,97.5


Showing details for Scenairo 2


Players not present in selected dataframe :  {'Kartik Tyagi'}
Enter team_name for Kartik Tyagi: rr
Enter category for Kartik Tyagi: bowl
Enter credits for Kartik Tyagi: 8
Selected Players DataFrame for current scenario : 


,team,player,category,credits
0,SRH,Abhishek Sharma,ALL,8
1,RR,Jofra Archer,BOWL,9
2,SRH,Kane Williamson,BAT,9.5
3,RR,Kartik Tyagi,BOWL,8
4,RR,Mahipal Lomror,ALL,8
5,SRH,Manish Pandey,BAT,9
6,SRH,Priyam Garg,BAT,8
7,RR,Rahul Tewatia,ALL,8.5
8,SRH,Rashid Khan,BOWL,9.5
9,SRH,Sandeep Sharma,BOWL,8


min_wk: 1, max_wk: 4
min_bat: 3, max_bat: 5
min_allr: 1, max_allr: 3
min_bowl: 3, max_bowl: 5


KeyError: "['WK'] not in index"